<div>
    <div style="float:left;">
        <img src="https://root.cern.ch/root/htmldoc/guides/users-guide/pictures/rootlogo.png" width="400px" />
    </div>
    <div style="float:left;">
        <img src="https://rawgit.com/qati/GSOC16/master/img/tmva-logo.svg" width="500px"/>
    </div>
</div>

<hr style="border-top-width: 4px; border-top-color: #34609b;">

<!--<script src="JsRoot/scripts/JSRootCore.js?jq2d&onload=JsRootLoadedCall" type="text/javascript"></script>-->

In [1]:
import ROOT
from ROOT import TFile, TMVA, TCut

Welcome to JupyROOT 6.09/01


## Enable JS visualization

In [2]:
%jsmva on

# Dataset infos

In [3]:
infname     = "tmva_class_example.root"
dataset     = "tmva_class_example"
treeNameSig = "TreeS"
treeNameBkg = "TreeB"
outfname    = "TMVA.root"

## Declare Factory and DataLoader

In [4]:
outputFile = TFile( outfname, 'RECREATE' )

TMVA.Tools.Instance();

In [5]:
factory = TMVA.Factory(JobName="TMVAClassification", TargetFile=outputFile,
            V=False, Color=False, DrawProgressBar=True, Transformations=["I", "D", "P", "G","D"],
                       AnalysisType="Classification")

In [6]:
loader = TMVA.DataLoader(dataset)

## Adding variables to DataLoader

In [7]:
loader.AddVariable( "myvar1 := var1+var2", 'F' )
loader.AddVariable( "myvar2 := var1-var2", "Expression 2", 'F' )
loader.AddVariable( "var3",                "Variable 3", 'F' )
loader.AddVariable( "var4",                "Variable 4", 'F' )

loader.AddSpectator( "spec1:=var1*2",  "Spectator 1",  'F' )
loader.AddSpectator( "spec2:=var1*3",  "Spectator 2",  'F' )

## If the dataset is not available on local computer we download from cern server

In [8]:
if ROOT.gSystem.AccessPathName( "./"+infname ) != 0: 
    ROOT.gSystem.Exec( "wget https://root.cern.ch/files/" + infname)

## Setting up dataset from Trees

In [9]:
input = TFile.Open( infname )

# Get the signal and background trees for training
signal      = input.Get( treeNameSig )
background  = input.Get( treeNameBkg )
    
# Global event weights (see below for setting event-wise weights)
signalWeight     = 1.0
backgroundWeight = 1.0

signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(signal, signalWeight)
loader.AddBackgroundTree(background, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = signal
loader.fTreeB = background

loader.PrepareTrainingAndTestTree(SigCut=mycuts, BkgCut=mycutb,
            nTrain_Signal=1000, nTrain_Background=1000, nTest_Signal=2000, nTest_Background=2000,
                                  SplitMode="Random", NormMode="NumEvents", V=False);

## Visualizing input variables

In [10]:
loader.DrawInputVariable("myvar1")

### We can also visualize transformations on input variables

In [11]:
loader.DrawInputVariable("myvar1", processTrfs=["D", "N"]) #I;N;D;P;U;G,D

## Correlation matrix of input variables

In [12]:
loader.DrawCorrelationMatrix("Signal")

In [13]:
factory.BookDNN(loader)

## Booking methods

In [14]:
factory.BookMethod( DataLoader=loader, Method=TMVA.Types.kSVM, MethodTitle="SVM", 
                Gamma=0.25, Tol=0.001, VarTransform="Norm" )

factory.BookMethod( loader,TMVA.Types.kMLP, "MLP", 
        H=False, V=False, NeuronType="tanh", VarTransform="N", NCycles=600, HiddenLayers="N+5",
                   TestRate=5, UseRegulator=False )

factory.BookMethod( loader,TMVA.Types.kLD, "LD", 
        H=False, V=False, VarTransform="None", CreateMVAPdfs=True, PDFInterpolMVAPdf="Spline2",
                   NbinsMVAPdf=50, NsmoothMVAPdf=10 )

factory.BookMethod( loader,TMVA.Types.kLikelihood,"Likelihood","NSmoothSig[0]=20:NSmoothBkg[0]=20:NSmoothBkg[1]=10",
    NSmooth=1, NAvEvtPerBin=50, H=True, V=False,TransformOutput=True,PDFInterpol="Spline2")

factory.BookMethod( loader, TMVA.Types.kBDT, "BDT",
    H=False, V=False, NTrees=850, MinNodeSize="2.5%", MaxDepth=3, BoostType="AdaBoost", AdaBoostBeta=0.5,
                   UseBaggedBoost=True, BaggedSampleFraction=0.5, SeparationType="GiniIndex", nCuts=20 )

<ROOT.TMVA::MethodBDT object ("BDT") at 0x7176530>

# Train Methods

In [15]:
factory.TrainAllMethods()

TFHandler_DNN
VariableMeanRMSMinMax
myvar10.0330140.38087-1.00001.0000
myvar20.0777380.30037-1.00001.0000
var30.113290.37003-1.00001.0000
var40.131010.42584-1.00001.0000
TFHandler_DNN
VariableMeanRMSMinMax
myvar10.0330140.38087-1.00001.0000
myvar20.0777380.30037-1.00001.0000
var30.113290.37003-1.00001.0000


## Testing the methods

In [16]:
factory.TestAllMethods()

## Evaluate the methods

In [17]:
factory.EvaluateAllMethods()

## Classifier Output Distributions

In [18]:
factory.DrawOutputDistribution(dataset, "MLP")

## Classifier Probability Distributions

In [19]:
factory.DrawProbabilityDistribution(dataset, "LD")

## ROC curve

In [20]:
factory.DrawROCCurve(dataset)

## Classifier Cut Efficiencies

In [21]:
factory.DrawCutEfficiencies(dataset, "MLP")

## Draw Neural Network

* Mouseover (node, weight): focusing
* Zooming and grab and move supported
* Reset: double click

In [22]:
factory.DrawNeuralNetwork(dataset, "MLP")

## Draw Deep Neural Network

In [23]:
factory.DrawNeuralNetwork(dataset, "DNN")

## Close the factory's output file

In [ ]:
outputFile.Close()